# Special Session: Noise Characterization and Error Mitigation in Near-Term Quanutm Computers

* Christopher J. Wood (cjwood@us.ibm.com)
* IBM Quantum, T. J. Watson Research Center, Yorktown Heights, USA

This notebook contains the accompanying code to the ICCD 2020 special session proceeding *Noise Characterization and Error Mitigation in Near-Term Quantum Computers*.

## 4.Measurement Error Mitigation

This notebook demonstrates performing measurement error mitigation on stabilizer graph state fidelity estimates on a 4-qubit subset of an IBM Quantum device using the continous-time Markov Process (CTPM) error mitigation technique.

*Note: this uses code in a development branch of qiskit-ignis. This branch can be installed using the `environment.yml` or `requirements.txt` file in this repository.*

In [ ]:
# Qiskit Module
import qiskit
import qiskit.quantum_info as qi
import qiskit.ignis.mitigation as mit

# Noisy Simulation
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.utils import insert_noise

# File utilities for saving and loading JSON data
from file_utils import save_to_json, load_properties, load_result

# Other
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Helper functions

The following helper functions are used to generate stabilzier measurement circuits and compute expectation values from result data.

In [ ]:
def stabilizer_measure_circuit(stabilizer, initial_circuit=None):
    """Return a stabilizer measurement cirucits.
    
    Args:
        stabilizer (str): a stabilizer string
        initial_circuit (QuantumCircuit): Optional, the initial circuit.
    
    Returns:
        QuantumCircuit: the circuit with stabilizer measurements.
    """
    if stabilizer[0] in ['+', '-']:
        stab = stabilizer[1:]
    else:
        stab = stabilizer

    if initial_circuit is None:
        circ = QuantumCircuit(len(stab))
    else:
        circ = initial_circuit.copy()

    for i, s in enumerate(reversed(stab)):
        if s == 'X':
            circ.h(i)
    circ.measure_all()
    return circ


def stabilizer_measure_diagonal(stabilizer):
    """Return the diagonal vector for a stabilizer measurement.
    
    Args:
        stabilizer (str): a stabilizer string
    
    Returns:
        np.ndarray: the diagonal for measurement in the stabilizer basis.
    """
    if stabilizer[0] in ['+', '-']:
        stab = stabilizer[1:]
    else:
        stab = stabilizer
        
    diag = np.array([1])
    for s in reversed(stab):
        if s == 'I':
            tmp = np.array([1, 1])
        else:
            tmp = np.array([1, -1])
        diag = np.kron(tmp, diag)
    return diag


def stabilizer_expvals(result, stabilizers, mitigator=None):
    """Compute expectation values from stabilizer measurement results."""

    stabilizer_diagonals = [
        stabilizer_measure_diagonal(stab) for stab in stabilizers]

    expvals_error = [
        mit.expectation_value(result.get_counts(i),
                              diagonal=stabilizer_measure_diagonal(stab),
                              mitigator=mitigator)
        for i, stab in enumerate(stabilizers)]

    return expvals_error

    
def stabilizer_fidelity(expvals):
    """Compute stabilizer state fidelity from stabilizer expvals."""
    mean = np.mean([i[0] for i in expvals])
    stddev = np.sqrt(np.sum([i[1] ** 2 for i in expvals]))
    return mean, stddev

## Graph State Circuit

We will estimate the state fidelity of preparing an entangled 4-qubit graph state by computing the expectation values of the graph state stabilizers. First we prepare a 4-qubit graph state quantum circuit

In [ ]:
num_qubits = 4
coupling_graph = [[0, 1], [1, 2], [2, 3]]

state_circuit = qiskit.QuantumCircuit(num_qubits)
state_circuit.h(range(num_qubits))
for i, j in coupling_graph:
    state_circuit.cz(i, j)
state_circuit.h(range(num_qubits))
state_circuit.draw('mpl').savefig('figures/graph_state4.png', dpi=150)
state_circuit.draw('mpl')

### Stabilizer Measurement Circuits

Next we compute the set of stabilizers for the graph state using the `Clifford` class from the `quantum_info` module, and then convert these into measurement circuits

In [ ]:
clifford = qi.Clifford(state_circuit)
stabilizers = clifford.stabilizer.pauli.to_labels()
print('Stabilizers:', stabilizers)

In [ ]:
stabilizer_circuits = [stabilizer_measure_circuit(stab, state_circuit)
                       for stab in stabilizers]

### Measurement Error Characterization Circuits

We also need to run the measurement error characterization circuits as outlined in the previous notebook.

In [ ]:
labels = ['0000', '0011', '0101', '1001', '0110', '1010', '1100', '1111']
meas_cal_circuits, metadata, _ = mit.MeasMitigatorGenerator(4, labels=labels).run()

### Run Experiment

Next execute the circuits on the IBM Quantum Boeblingen backend. You may change this to run on any IBM Quantum backend you have access to.

In [ ]:
use_saved_data = True
file_properties = 'data/properties_stab_v1.json'
file_result = 'data/result_stab_v1.json'
file_result_mit = 'data/result_stab_mit_v1.json'

if use_saved_data:

    # Load saved result data from file
    result_backend = load_result(file_result)
    result_mit_backend = load_result(file_result_mit)

    # Load saved properties from file
    properties = load_properties(file_properties)

else:

    # Load IBMQ Account data
    from qiskit import IBMQ
    IBMQ.load_account()

    # Get backend for experiment
    provider = IBMQ.get_provider(project='ignis')
    backend = provider.get_backend('ibmq_boeblingen')

    # Run QPT on backend
    shots = 8192
    job_mit_backend = qiskit.execute(meas_cal_circuits, backend, shots=shots)
    job_backend = qiskit.execute(stabilizer_circuits, backend, shots=shots)
    print('Job IDs: {}'.format([job_mit_backend.job_id(), job_backend.job_id()]))

    # Save a copy of the backend properties to file
    properties = backend.properties()
    save_to_json(properties, file_properties)

    # Retrieve results (this may take a while depending on the queue)
    result_mit_backend = job_mit_backend.result()
    result_backend = job_backend.result()
    
    # Save a copy of the result data to file
    save_to_json(result_backend, file_result)    
    save_to_json(result_mit_backend, file_result_mit)    

### Run Simulation

Next we run the characterization experiment using the Qiskit Aer simulator with the basic device noise model based on the backend used to run the experiment

In [ ]:
# Simulator backend
simulator = QasmSimulator()
shots = 8192
seed_simulator = 1984

# Noise Model
noise_model = NoiseModel.from_backend(properties, warnings=False)

# Run simulation
result_sim = qiskit.execute(stabilizer_circuits,
                            simulator,
                            shots=shots,
                            seed_simulator=seed_simulator,
                            basis_gates=noise_model.basis_gates,
                            noise_model=noise_model).result()

result_mit_sim = qiskit.execute(meas_cal_circuits,
                                simulator,
                                shots=shots,
                                seed_simulator=seed_simulator,
                                basis_gates=noise_model.basis_gates,
                                noise_model=noise_model).result()

## Measurement Error Mitigated Expectation Values

Next we use our helper functions to compute the exectation values for each stabilzier measurement with and without using the measurement error mitigator. This makes use of the `expectation_value` function from the `mitigation` module.

In [ ]:
# Fit measurement error mitigators
mitigator_backend = mit.MeasMitigatorFitter(result_mit_backend, metadata).fit()
mitigator_sim = mit.MeasMitigatorFitter(result_mit_sim, metadata).fit()

# Unmitigated Expectation Values 
expvals_nomit_backend = stabilizer_expvals(result_backend, stabilizers)
expvals_nomit_sim = stabilizer_expvals(result_sim, stabilizers)

# Measurement error mitigated expectation values
expvals_mit_backend = stabilizer_expvals(result_backend, stabilizers,
                                         mitigator=mitigator_backend)
expvals_mit_sim = stabilizer_expvals(result_sim, stabilizers,
                                     mitigator=mitigator_sim)

### Stabilizer Expectation Value Plot

In [ ]:
vals_nomit_b = [i[0] for i in expvals_nomit_backend]
vals_mit_b = [i[0] for i in expvals_mit_backend]
vals_nomit_s = [i[0] for i in expvals_nomit_sim]
vals_mit_s = [i[0] for i in expvals_mit_sim]

colors1 = sns.color_palette("BuGn_r")
colors2 = sns.color_palette("GnBu_d")

labels = stabilizers
x = np.arange(len(labels))
width = 0.2

label_nomit_b = 'Unmitigated (experiment)'
label_mit_b = 'Mitigated (experiment)'
label_nomit_s = 'Unmitigated (simulated)'
label_mit_s = 'Mitigated (simulated)'

fig, ax = plt.subplots()
ax.bar(x - 3 * width / 2, vals_nomit_b, width, label=label_nomit_b, color=colors1[0])
ax.bar(x - width / 2, vals_mit_b, width, label=label_mit_b, color=colors1[2])
ax.bar(x + width / 2, vals_nomit_s, width, label=label_nomit_s, color=colors2[0])
ax.bar(x + 3 * width / 2, vals_mit_s, width, label=label_mit_s, color=colors2[2])

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel(r'Expectation Value $\langle S\rangle$', fontsize=14)
ax.set_xlabel('Stabilizer', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(labels, fontsize=14)
ax.set_ylim(0.5, 1.05)
ax.legend(loc='lower right', fontsize=13, framealpha=1)
fig.savefig('figures/expval_mit.png', dpi=150)

## Measurement Error Mitigated State Fidelity

Finally we can use the mitigated expectation values to estimate the error mitigated state fidelity of the prepared graph state

In [ ]:
F_nomit_backend = stabilizer_fidelity(expvals_nomit_backend)
F_nomit_sim = stabilizer_fidelity(expvals_nomit_sim)
F_mit_backend = stabilizer_fidelity(expvals_mit_backend)
F_mit_sim = stabilizer_fidelity(expvals_mit_sim)

print('Graph-state fidelity estimates')
print('\nNo mitigation')
print('F({}) = {:.3f} \u00B1 {:.3f}'.format(
    properties.backend_name, F_nomit_backend[0], F_nomit_backend[1]))
print('F(simulated) = {:.3f} \u00B1 {:.3f}'.format(
    F_nomit_sim[0], F_nomit_sim[1]))

print('\nCTMP error mitigation')
print('F({}) = {:.3f} \u00B1 {:.3f}'.format(
    properties.backend_name, F_mit_backend[0], F_mit_backend[1]))
print('F(simulated) = {:.3f} \u00B1 {:.3f}'.format(
    F_mit_sim[0], F_mit_sim[1]))

### Exact state fidelity for simulated case

For the simualted case we can compare the mitigated fidelity estimate to the exact value computed from the device noise model without measurement error

In [ ]:
state_circuit_noise = insert_noise(state_circuit, noise_model, transpile=True)
ideal_state = qi.Statevector.from_instruction(state_circuit)
noisy_state = qi.DensityMatrix.from_instruction(state_circuit_noise)
F_exact_sim = qi.state_fidelity(noisy_state, ideal_state)

print('Exact State Fidelity')
print('F(simulated) = {:.3f}'.format(F_exact_sim))

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table